<a href="https://colab.research.google.com/github/Kogo0813/Windroad8/blob/main/windpredict_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!echo $COLAB_TPU_ADDR #코랩에서 런타임->런타임 유형 변경 -> TPU로 변경

10.92.187.138:8470


In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from osgeo import gdal

# TPU gRPC 접근 URI
TPU_PATH = f"grpc://{os.environ['COLAB_TPU_ADDR']}"
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=TPU_PATH)
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [4]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive
%cd MyDrive
%cd data_from_s

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive
/content/drive/MyDrive
/content/drive/MyDrive/data_from_s


In [6]:
## 데이터 불러오는 부분 주석 확인해서 전체데이터로 바꿔주세요
X = []
Y = []
for i in range(1, 2): #전체 데이터를 돌릴땐 2->12로 수정해주세요
  for h in range(1, 3): # 마찬가지 3->4 수정필요
    for s in range(0,2): # 2->6 수정필요
      for d in range(0, 180, 60): #180 ->360 수정 필요
        if i==3 and h==1 and s ==3 : 
          continue
        if i==3 and h==3 and s ==1 and d==300: 
          continue
        if i==3 and h==3 and s ==5 and d==120: 
          continue
        if i==8 and h==1 and s ==3 and d==120: 
          continue
        if i==9 and h==2 and s ==3 and d==300: 
          continue ## 이 파일들 이상해서 그냥 안쓸려고 합니다.
        landuse = gdal.Open("입력데이터/"+"Landuse_"+"{:006d}".format(i)+".tif")
        landuse = np.array(landuse.GetRasterBand(1).ReadAsArray())
        landuse = landuse[:1200,:1500]
        terrain = gdal.Open("입력데이터/"+"Terrain_"+"{:006d}".format(i)+".tif")
        terrain = np.array(terrain.GetRasterBand(1).ReadAsArray())
        terrain = terrain[:1200,:1500]
        stacked_layer_input = []
        stacked_layer_output = []
        stacked_layer_input.append(np.dstack((landuse,terrain)))
        for j in range(360, 3960, 360):
            uz = np.loadtxt("0000" +"{:02d}".format(i) +"_R010_H0"+ str(h) +".0_S00"+str(s)+".00_D"+"{:003d}".format(d)+'_uz00'+"{:04d}".format(j)+'.dw', skiprows = 8, dtype = 'int', encoding='latin-1')
            vz = np.loadtxt("0000" +"{:02d}".format(i) +"_R010_H0"+ str(h) +".0_S00"+str(s)+".00_D"+"{:003d}".format(d)+'_vz00'+"{:04d}".format(j)+'.dw', skiprows = 8, dtype = 'int', encoding='latin-1')
            stacked_layer_output.append(np.dstack((uz,vz)))
        X.append(stacked_layer_input)
        Y.append(stacked_layer_output)

In [8]:
X = np.array(X)
Y = np.array(Y)
X.shape

(12, 1, 1200, 1500, 2)

In [9]:
grid_size = (60, 75) #이미지 나누는 부분 이부분을 1200과 1500의 약수로 바꿔가면서 테스트

X_grid = np.empty((X.shape[0] * int(X.shape[2]/grid_size[0]) * int(X.shape[3]/grid_size[1]),1,grid_size[0],grid_size[1],2))
Y_grid = np.empty((Y.shape[0]* int(X.shape[2]/grid_size[0]) * int(X.shape[3]/grid_size[1]),10,grid_size[0],grid_size[1],2))

index = 0
for k in range(X.shape[0]):
    image = X[k, 0, :, :, :]
    for i in range(0, image.shape[0], grid_size[0]):
        for j in range(0, image.shape[1], grid_size[1]):
            cell = image[i:i+grid_size[0], j:j+grid_size[1], :]
            X_grid[index] = cell.reshape(1,grid_size[0],grid_size[1],2)
            index += 1

index = 0
for k in range(Y.shape[0]):
    image = Y[k, :, :, :, :]
    for i in range(0, image.shape[1], grid_size[0]):
        for j in range(0, image.shape[2], grid_size[1]):
            cell = image[:,i:i+grid_size[0], j:j+grid_size[1], :]
            Y_grid[index] = cell.reshape(10,grid_size[0],grid_size[1],2)
            index += 1

In [10]:
X_grid.shape

(4800, 1, 60, 75, 2)

In [1]:
plt.figure(figsize=(2,3))#landuse
plt.imshow(X_grid[1,0,:,:,0])
plt.show()
plt.figure(figsize=(2,3)) #terrain
plt.imshow(X_grid[1,0,:,:,1])
plt.show()
plt.figure(figsize=(20,30)) 
for i in range(Y_grid.shape[1]): #uz
    plt.subplot(1,10,i+1)
    plt.imshow(Y_grid[1,i,:,:,0])
plt.show()
plt.figure(figsize=(20,30)) 
for i in range(Y_grid.shape[1]): #vz
    plt.subplot(1,10,i+1)
    plt.imshow(Y_grid[1,i,:,:,1])
plt.show()
plt.figure(figsize=(20,30)) 

NameError: ignored

In [11]:
param = []
for i in range(1, 2): #전체 데이터를 돌릴땐 2->12로 수정해주세요
  for h in range(1, 3): # 마찬가지 3->4 수정필요
    for s in range(0,2): # 2->6 수정필요
      for d in range(0, 180, 60): #180 ->360 수정 필요
        if i==3 and h==1 and s ==3 : 
          continue
        if i==3 and h==3 and s ==1 and d==300: 
          continue
        if i==3 and h==3 and s ==5 and d==120: 
          continue
        if i==8 and h==1 and s ==3 and d==120: 
          continue
        if i==9 and h==2 and s ==3 and d==300: 
          continue 
        for k in range(0,400) :
          height = np.full((60, 75), h)
          speed = np.full((60,75), s)
          direction = np.full((60,75), d)
          param.append(np.dstack((height, speed, direction)))

In [12]:
param = np.array(param)

In [13]:
X_grid = X_grid.reshape(4800,60,75,2)

In [14]:
X_final = np.concatenate([param, X_grid], axis=-1) 

In [15]:
Y_final = Y_grid[:,0,:,:,:]
Y_final.shape

(4800, 60, 75, 2)

In [16]:
#Terrain, Landuse와 세 개의 정수데이터를 60*75 배열로 채워 합친뒤 이를 입력으로 받아 첫 번째 uz,vz를 만드는 CNN 모델
#Terrain, Landuse 두개의 이미지를 conv2d하고 flatten한 뒤 정수 입력변수 3개를 fullyconneted한 후 다시 업샘플링 하는 방식도 시도해 볼만해보임
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Sequential, Model
import tensorflow.keras.backend as K
from keras.utils import get_custom_objects
with strategy.scope():
  def r2_metric(y_true, y_pred):
      SS_res =  K.sum(K.square(y_true - y_pred))
      SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
      return (1 - SS_res/(SS_tot + K.epsilon()))

  get_custom_objects().update({'r2_metric': r2_metric})

  model = Sequential()
  model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(60, 75, 5)))
  model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
  model.add(Conv2D(2, (3, 3), activation='sigmoid', padding='same'))

  model.compile(optimizer='adam', loss='mse',metrics='r2_metric')
  model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 60, 75, 64)        2944      
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 75, 32)        18464     
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 75, 2)         578       
                                                                 
Total params: 21,986
Trainable params: 21,986
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#코랩 세션 다운... 샘플 수가 많아서 그런것으로 생각 됨
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(X_final, Y_final, test_size=0.2, shuffle=True)
with strategy.scope():
  early_stop = EarlyStopping(monitor='val_loss',min_delta=0.001, patience=3, verbose=1, mode='min',restore_best_weights=True)
  history = model.fit(x=x_train, y=y_train, validation_data=(x_val, y_val), epochs=400, callbacks=[early_stop])